In [1]:
'''Projet PYTHON PCO''' 
'''CHARPIAT Arthur, LANCIAUX Jérôme'''
'''L2 INFO 2'''
import tkinter as tk
import json
from tkinter import *


# --------------------------------------------------------------------------------------------------------------------------#
# -----------------------------------------------------SpaceInvaders-------------------------------------------------------------#
# --------------------------------------------------------------------------------------------------------------------------#
'''Classe Principale, porte la fenêtre et la frame'''
class SpaceInvaders(object):

    # Main game class
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Space Invaders")
        self.frame = tk.Frame(self.root, width=400, height=400)
        self.frame.pack(side="top", fill="both", expand=True)
        self.game = Game(self.frame)
    '''Fonction qui démarre le jeu, appelée à la fin'''
    def play(self):
        if self.game.status == 1 :
            self.game.start_animation()
            self.root.bind("<Key>", self.game.defender.controle)
            self.root.mainloop()


# --------------------------------------------------------------------------------------------------------------------------#
# -----------------------------------------------------GAME-----------------------------------------------------------------#
# --------------------------------------------------------------------------------------------------------------------------#
'''Cette classe va construire la frame et gérer tout les élements relatifs aux différentes classes du jeu'''
class Game(object):
    def __init__(self, frame):
        self.frame = frame
        '''Création du canvas'''
        self.canvas_width = 600
        self.canvas_height = 600
        self.canvas = tk.Canvas(self.frame, width=self.canvas_width, height=self.canvas_height, bg="black")
        self.canvas.pack(side="top", fill="both", expand=True)
        self.defender = Defender()
        self.defender.install_in(self.canvas)
        self.fleet = Fleet()
        self.fleet.install_in(self.canvas)
        self.timer = 0
        '''Tentative d'ajout de score... Non fonctionnelle'''
        self.score = Score("Jack",self.fleet.add_score,self.timer)
        '''Image de partie perdue'''
        self.game_over = tk.PhotoImage(file='D:/IMGRANDOM/gameover.gif')
        '''Image de partie gagnée'''
        self.win = tk.PhotoImage(file='D:/IMGRANDOM/win.gif')
        '''Permet de gérer les états du jeu'''
        self.status = 1
       
        '''Fenêtre du score, avec un Label'''
        self.gscore = Label(self.canvas,text='Score : {}'.format(self.fleet.add_score)  , bg="Black", fg="White", font="Impact")
        self.canvas.create_window(60, 20, window=self.gscore)
        
        '''Fenêtre du temps, avec un Label'''
        self.gtime = Label(self.canvas,text='Temps : {}'.format(self.timer)  , bg="Black", fg="White", font="Impact")
        self.canvas.create_window(self.canvas_width//2,20, window=self.gtime)
        
        

    '''Récupère le move_in de la classe Fleet'''    
    def move_aliens_fleet(self):
        self.fleet.move_in(self.canvas)
    
    '''Fonction très importante qui va permettre d'animer le canvas, et d'actualiser en permanence les données'''
    def animation(self):
        '''Actualise le score et le temps'''
        self.gscore.config(text ='Score : {}'.format(self.fleet.add_score))
        self.timer +=0.05
        self.gtime.config(text ='Temps : {}'.format('%d' % self.timer))
        
        '''Lorsque la taille de la Fleet est de 0, partie gagnée'''
        if len(self.fleet.aliens_fleet) < 1:
            self.status = 2
            self.canvas.create_image(self.canvas_width // 2, self.canvas_height // 2, image=self.win)
            
        else :
            '''Récupération des différentes coordonnées des élements '''
            x1,y1,x2,y2 = self.canvas.bbox('alien')
            xd1,yd1,xd2,yd2 = self.canvas.bbox('defender')
            
            '''La partie continue tant qu'aucune autre conditions n'est activée'''
            if self.status == 1:
                self.move_aliens_fleet()
                self.canvas.after(50, self.animation)
            '''Si le nombre de bullet est supérieur à 0, début animation bullet'''
            if (len(self.defender.fired_bullets)>0):
                self.move_bullets()
                self.fleet.manage_touched_aliens_by(self.canvas, self.defender)
                
            '''Si la fleet touche le haut du defender, partie perdue'''
            if y2 > yd1 and self.status == 1 :
                self.score.toFile("D:/IMGRANDOM/score.json")
                self.canvas.create_image(self.canvas_width // 2, self.canvas_height // 2, image=self.game_over)
                self.status = 0
                
                
    '''Démarre l animation'''
    def start_animation(self):
        if self.status == 1:
            self.canvas.after(10, self.animation)
    '''Applique le mouvement de la bullet à toutes les bullets sur le canvas'''
    def move_bullets(self):
        for bullet in self.defender.fired_bullets:
            bullet.move_in(self.canvas)

# --------------------------------------------------------------------------------------------------------------------------#
# -----------------------------------------------------DEFENDER-------------------------------------------------------------#
# --------------------------------------------------------------------------------------------------------------------------#
'''Classe relative au defender, avec les contrôles de celui-ci'''
class Defender(object):
    def __init__(self):
        self.width = 20
        self.height = 20
        self.speed = 20
        self.id = None
        self.max_fired_bullets = 8
        self.canvas=None
        self.fired_bullets=[]
    
    '''Création du defender'''

    def install_in(self, canvas):
        max_w = int(canvas.cget("width"))
        max_h = int(canvas.cget("height"))
        self.canvas=canvas
        self.id = canvas.create_rectangle(max_w // 2, max_h*0.9, max_w // 2+self.width, max_h*0.9+self.height, fill="white", tags ="defender")

    '''Gestion des deplacements et des tirs du defender'''

    def move_in(self, canvas, dx):
        x1, y1, x2, y2 = canvas.bbox(self.id)
        if x1<10:
            if dx<0:
                dx=-dx
        elif x2>int(self.canvas.cget("width"))-10:
            if dx > 0:
                dx = -dx
        self.canvas.move(self.id,dx,0)

    '''Gestion des deplacements'''
    
    def controle(self,event):
        if event.keysym == "space":
            self.fire()
        if event.keysym == "Right":
            self.move_in(self.canvas,self.speed)
        if event.keysym == "Left":
            self.move_in(self.canvas, -self.speed)
            
    '''Gestion des tirs du defender, si plus de 8 bullets sur le canvas, impossible de tirer davantage'''
    
    def fire(self):
        x1, y1, x2, y2 = self.canvas.bbox(self.id)
        if len(self.fired_bullets)<self.max_fired_bullets:
            fire_bullet=Bullet(self)
            self.fired_bullets.append(fire_bullet)
            fire_bullet.install_in(self.canvas)


# --------------------------------------------------------------------------------------------------------------------------#
# -----------------------------------------------------ALIEN----------------------------------------------------------------#
# --------------------------------------------------------------------------------------------------------------------------#
'''Classe gérant la création, les déplacements ainsi que les collisions d un alien'''
class Alien(object):
    def __init__(self):
        self.id = None
        self.alive = True
    image_alien = None
    
    '''Création d un alien'''
    
    def install_in(self, canvas, x, y, tag):
        self.id = canvas.create_image(x, y, image=Alien.get_image(), tags=tag)
        
    '''Déplacement d un alien'''
    
    def move_in(self, canvas, dx, dy):
        canvas.move(self.id, dx, dy)
        
    '''Lorsqu un alien et une bullet entre en collision, alien déclaré mort '''
    
    def touched_by(self, canvas, projectile):
        x1, y1, x2, y2 = canvas.bbox(self.id)
        '''Ici, touch va prendre "l'ID" des éléments ayant les même coordonnées sur le canvas'''
        touch=canvas.find_overlapping(x1, y1, x2, y2)
        '''Si c'est le cas, l'alien passe en mode "mort"'''
        if touch[len(touch)-1]==projectile.id:
            self.alive=False
            
    '''Methode de classe qui récupère l image'''
    
    @classmethod
    def get_image(cls):
        if cls.image_alien == None:
            cls.image_alien = tk.PhotoImage(file='D:/IMGRANDOM/alien.gif')
        return cls.image_alien




# --------------------------------------------------------------------------------------------------------------------------#
# -----------------------------------------------------BULLET----------------------------------------------------------------#
# --------------------------------------------------------------------------------------------------------------------------#
'''La classe bullet introduit la création d'une balle, les déplacement de celle-ci ainsi que son effacement si elle sort du canvas'''
class Bullet(object):
    def __init__(self, shooter):
        self.radius = 5
        self.color = "red"
        self.speed = 8
        self.id = None
        self.shooter = shooter

    '''Creation de la bullet'''
    
    def install_in(self, canvas):
        x1, y1, x2, y2 = canvas.bbox(self.shooter.id)
        self.id=canvas.create_oval(x1+5, y1+5, x2-5, y2-5, fill=self.color)
        
    '''Deplacement de la bullet, et effacement si elle depasse le canvas'''
    
    def move_in(self, canvas):
        x1, y1, x2, y2 = canvas.bbox(self.id)
        if y1>=0:
            canvas.move(self.id, 0, -self.speed)
        else:
            self.shooter.fired_bullets.remove(self)
            canvas.delete(self.id)



# --------------------------------------------------------------------------------------------------------------------------#
# -----------------------------------------------------FLEET----------------------------------------------------------------#
# --------------------------------------------------------------------------------------------------------------------------#
'''Classe appliquant les autres classe à plus grande échelle, gestion des objets dans le canvas '''
class Fleet(object):
    def __init__(self):
        '''Détermine la taille et la vitesse de la Fleet'''
        self.aliens_lines = 5
        self.aliens_columns = 10
        self.aliens_inner_gap = 50
        self.alien_x_delta = 1
        self.alien_y_delta = 5
        fleet_size =self.aliens_lines * self.aliens_columns
        self.aliens_fleet = [None] * fleet_size
        self.direction=">" 
        self.add_score = 0
        self.explosion = tk.PhotoImage(file='D:/IMGRANDOM/explo.gif')
    '''Création de la Fleet d Aliens'''
    
    def install_in(self, canvas):
        x=100
        y=100
        for i in range(self.aliens_lines):
            for j in range(self.aliens_columns):
                '''Applique install in de l'alien à tout le contenu de self.aliens_fleet'''
                self.aliens_fleet[(i*self.aliens_columns)+j]=Alien()
                self.aliens_fleet[(i*self.aliens_columns)+j].install_in(canvas,x, y,'alien')
             
                x+=self.aliens_inner_gap
            '''A chaque fin de ligne, x reviens à la position de départ et les aliens sont crées sur une nouvelle ligne'''
            y+=self.aliens_inner_gap
            x=100
    '''Deplacement de la Fleet dans le Canvas'''
        
    def move_in(self,canvas):
        x1, y1, x2, y2 = canvas.bbox("alien")
        
        '''Si la fleet touche le bord gauche, elle repart vers la droite et descend'''
        if x1<10 and (self.direction=="<"): 
            self.direction=">"
            self.alien_x_delta = -self.alien_x_delta 
            for alien in self.aliens_fleet:
                alien.move_in(canvas,0,self.alien_y_delta)
                
        '''Si la fleet touche le bord droit, elle repart vers la gauche et descend'''
        if (x2>int(canvas.cget("width"))-10) and (self.direction==">"): 
            self.direction="<"
            self.alien_x_delta = -self.alien_x_delta
            for alien in self.aliens_fleet:
                alien.move_in(canvas,0,self.alien_y_delta)
        for alien in self.aliens_fleet: 
            alien.move_in(canvas,self.alien_x_delta, 0)
            
    '''Gestion des items dans le canvas, effacement des bullets / aliens'''        

    def manage_touched_aliens_by(self,canvas,defender):
        for bullet in defender.fired_bullets:
            for alien in self.aliens_fleet:
                alien.touched_by(canvas, bullet)
                if alien.alive==False:
                    defender.fired_bullets.remove(bullet)
                    canvas.delete(bullet.id)
                    self.aliens_fleet.remove(alien)
                    self.add_score += 20
                    canvas.itemconfigure(alien.id , state ='hidden')
                       
# --------------------------------------------------------------------------------------------------------------------------#
# -----------------------------------------------------START----------------------------------------------------------------#
# --------------------------------------------------------------------------------------------------------------------------#
SpaceInvaders().play()